# Latent Dirichlet Allocation

## Data

NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv('npr.csv')

In [3]:
len(npr)

11992

In [4]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

max_df is used for removing terms that appear too frequently, also known as "corpus-specific stop words". For example:

max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".
max_df = 25 means "ignore terms that appear in more than 25 documents".
The default max_df is 1.0, which means "ignore terms that appear in more than 100% of the documents". Thus, the default setting does not ignore any terms.

# -------------------------------------------------------------------------
min_df is used for removing terms that appear too infrequently. For example:

min_df = 0.01 means "ignore terms that appear in less than 1% of the documents".
min_df = 5 means "ignore terms that appear in less than 5 documents".
The default min_df is 1, which means "ignore terms that appear in less than 1 document". Thus, the default setting does not ignore any terms.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = cv.fit_transform(npr['Article'])

In [8]:
dtm = cv.fit_transform(npr['Article'])

In [9]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

## LDA

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = cv.fit_transform(npr['Article'])

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

In [12]:
LDA = LatentDirichletAllocation(n_components=7,random_state=42)

In [ ]:
# This can take awhile, we're dealing with a large amount of documents!
LDA.fit(dtm)

In [ ]:
topic_results = LDA.transform(dtm)
npr['Topic'] = topic_results.argmax(axis=1)

## Showing Stored Words

In [14]:
len(cv.get_feature_names())

54777

In [15]:
import random

In [16]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(cv.get_feature_names()[random_word_id])

meriting
tuberculosis
predisposed
supports
dramas
fortified
confrontations
wansink
interlaced
horsley


In [17]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(cv.get_feature_names()[random_word_id])

moussa
clampdown
tranche
mighty
sepe
tanimoto
republicanism
disemboweling
regret
underdiagnosed


### Showing Top Words Per Topic

In [18]:
len(LDA.components_)

7

In [19]:
LDA.components_

array([[2.55030234e+01, 1.18160521e+03, 1.42857161e-01, ...,
        1.42942888e-01, 1.42865635e-01, 1.42994709e-01],
       [1.75938770e+01, 7.79163845e+00, 1.42871959e-01, ...,
        1.42857152e-01, 1.43057304e-01, 1.42990129e-01],
       [5.56830532e+00, 7.61579261e+02, 1.42857370e-01, ...,
        4.84616992e+00, 2.46543716e+00, 1.42942152e-01],
       ...,
       [1.43329722e-01, 6.12135291e+02, 1.42857193e-01, ...,
        1.42868589e-01, 1.42859011e-01, 1.42897792e-01],
       [3.89096469e+01, 1.97272208e+01, 4.00585507e+00, ...,
        1.43174190e-01, 1.45396934e-01, 2.17293923e+00],
       [5.82136826e+00, 1.76095158e+03, 1.43163938e-01, ...,
        1.42994603e-01, 1.43326983e-01, 1.42965349e-01]])

In [20]:
len(LDA.components_[0])

54777

In [21]:
single_topic = LDA.components_[0]

In [23]:
single_topic.shape

(54777,)


[22,55,11].argsort()

[2,0,1]


In [24]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([49163, 21347, 20789, ..., 37374, 42993, 42561], dtype=int64)

In [25]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([ 9511, 49613, 53019, 11657, 21228, 40955, 36283, 37374, 42993,
       42561], dtype=int64)

In [26]:
# Word least representative of this topic
single_topic[9511]

2173.553334803384

In [27]:
# Word most representative of this topic
single_topic[42561]

7280.693330545898

In [48]:
single_topic.shape

(54777,)

In [28]:
single_topic = LDA.components_[0]
top_word_indices = single_topic.argsort()[-25:]

In [29]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

security
isis
officers
group
syria
international
say
years
npr
reported
state
attack
killed
according
military
city
told
war
country
government
reports
people
police
says
said


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [30]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['state', 'attack', 'killed', 'according', 'military', 'city', 'told', 'war', 'country', 'government', 'reports', 'people', 'police', 'says', 'said']


THE TOP 15 WORDS FOR TOPIC #1
['russian', 'npr', 'fbi', 'did', 'clinton', 'told', 'media', 'white', 'campaign', 'russia', 'house', 'news', 'president', 'said', 'trump']


THE TOP 15 WORDS FOR TOPIC #2
['world', 'year', 'make', 'day', 'home', 'time', 'city', 'new', 'years', 'just', 'people', 'water', 'like', 'food', 'says']


THE TOP 15 WORDS FOR TOPIC #3
['women', 'don', 'just', 'research', 'like', 'university', 'children', 'percent', 'care', 'study', 'students', 'school', 'health', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #4
['vote', 'court', 'new', 'percent', 'republican', 'law', 'campaign', 'obama', 'states', 'people', 'president', 'clinton', 'state', 'said', 'trump']


THE TOP 15 WORDS FOR TOPIC #5
['ve', 'going', 'years', 'don', 'life', 'new', 'really', 'way', 'says', 'know', 'think', 'time', 'pe

### Attaching Discovered Topic Labels to Original Articles

In [31]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [32]:
dtm.shape

(11992, 54777)

In [33]:
len(npr)

11992

In [34]:
topic_results = LDA.transform(dtm)


In [35]:
topic_results.shape

(11992, 7)

In [36]:
topic_results[0]

array([7.31632607e-02, 6.75553233e-01, 6.25535235e-03, 2.25570582e-04,
       2.21208569e-01, 2.25552570e-04, 2.33684623e-02])

In [37]:
topic_results[1].round(2)

array([0.24, 0.46, 0.  , 0.  , 0.14, 0.  , 0.16])

In [38]:
topic_results[0].argmax()

1

In [41]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 3, 4, 4], dtype=int64)

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [ ]:
npr.head()

In [ ]:
import numpy as np

In [ ]:
topic_results.argmax(axis=1)

In [ ]:
type(topic_results)

In [ ]:
topic_results.shape

In [43]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [44]:
npr['Topic'] = topic_results.argmax(axis=1)

In [45]:
npr.head(10)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
4,"From photography, illustration and video, to d...",5
5,I did not want to join yoga class. I hated tho...,3
6,With a who has publicly supported the debunk...,3
7,"I was standing by the airport exit, debating w...",2
8,"If movies were trying to be more realistic, pe...",2
9,"Eighteen years ago, on New Year’s Eve, David F...",2


In [47]:
npr[npr.Topic==1]

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
19,"In November, the typically straitlaced Office ...",1
31,Editor’s note: This post includes language tha...,1
59,There are a lot of reasons victims of sexual a...,1
62,Some prominent conservatives have signed on to...,1
71,Fox News anchor Megyn Kelly is leaving the cab...,1
85,"Rex Tillerson, Donald Trump’s nominee for U....",1
